In [ ]:
!pip install gradio requests

In [ ]:
import gradio as gr
import requests
import json

def check_environmental_relevance(api_key, user_message):
    """Check if the question is related to Environmental Studies"""
    url = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    check_prompt = f"""You are a filter that determines if a question is related to Environmental Studies.

Environmental Studies topics include:
- Ecology and ecosystems
- Climate change and global warming
- Pollution (air, water, soil, noise)
- Conservation and biodiversity
- Renewable and non-renewable resources
- Waste management and recycling
- Sustainable development
- Environmental policies and laws
- Deforestation and afforestation
- Wildlife and habitat protection
- Carbon footprint and greenhouse gases
- Water conservation
- Environmental impact assessment
- Green technology
- Natural disasters and environmental hazards

Question: "{user_message}"

Respond with ONLY "YES" if this question is related to Environmental Studies, or "NO" if it is not related to Environmental Studies."""

    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": [{"role": "user", "content": check_prompt}],
        "temperature": 0.1,
        "max_tokens": 10
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        answer = result['choices'][0]['message']['content'].strip().upper()
        return "YES" in answer
    except Exception as e:
        return False

def chat_with_bot(message, history, api_key):
    """Main chat function"""
    if not api_key or api_key.strip() == "":
        return history + [[message, "⚠️ Please enter your Groq API key to continue."]]

    # Check if question is related to Environmental Studies
    is_relevant = check_environmental_relevance(api_key, message)

    if not is_relevant:
        response = "🌿 I'm sorry, but I can only answer questions related to **Environmental Studies**. Please ask me about topics like climate change, pollution, ecosystems, conservation, sustainability, renewable energy, or other environmental topics."
        return history + [[message, response]]

    # If relevant, proceed with the actual response
    url = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Build conversation history for context
    messages = [
        {"role": "system", "content": """You are an expert Environmental Studies teacher. You provide EXTREMELY CONCISE answers - respond in a single word or a single short sentence (maximum 15 words).

IMPORTANT: Keep answers as brief as possible while being accurate.

Topics you cover:
- Ecology and ecosystems
- Climate change and global warming
- Pollution and its types
- Conservation and biodiversity
- Renewable and non-renewable resources
- Waste management
- Sustainable development
- Environmental policies
- Wildlife protection
- Green technology

Answer in ONE LINE or ONE WORD only."""}
    ]

    # Add conversation history
    for h in history:
        messages.append({"role": "user", "content": h[0]})
        messages.append({"role": "assistant", "content": h[1]})

    # Add current message
    messages.append({"role": "user", "content": message})

    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": messages,
        "temperature": 0.3,
        "max_tokens": 50
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        bot_response = result['choices'][0]['message']['content']
        return history + [[message, bot_response]]
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            error_msg = "❌ Invalid API key. Please check your Groq API key and try again."
        else:
            error_msg = f"❌ Error: {str(e)}"
        return history + [[message, error_msg]]
    except Exception as e:
        return history + [[message, f"❌ An error occurred: {str(e)}"]]

# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🌍 Environmental Studies Chatbot

        Welcome! I'm your Environmental Studies assistant. I can help you with questions about:
        - 🌡️ Climate Change & Global Warming
        - 🏭 Pollution & Waste Management
        - 🌳 Ecosystems & Biodiversity
        - ♻️ Sustainability & Conservation
        - ⚡ Renewable Energy
        - 🌊 Water & Air Quality
        - And much more!

        **Note:** I only answer questions related to Environmental Studies.
        """
    )

    with gr.Row():
        api_key_input = gr.Textbox(
            label="🔑 Groq API Key",
            placeholder="Enter your Groq API key here...",
            type="password",
            scale=4
        )
        gr.Markdown(
            """
            <div style="padding: 10px; font-size: 12px;">
            Don't have an API key? <br>
            Get one at <a href="https://console.groq.com/" target="_blank">console.groq.com</a>
            </div>
            """
        )

    chatbot = gr.Chatbot(
        label="Chat History",
        height=400,
        show_copy_button=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your Message",
            placeholder="Ask me anything about Environmental Studies...",
            scale=4
        )
        send_btn = gr.Button("Send 🚀", scale=1)

    clear = gr.Button("Clear Chat 🗑️")

    gr.Examples(
        examples=[
            "What is global warming and what causes it?",
            "Explain the greenhouse effect",
            "What are the main types of pollution?",
            "How does deforestation affect biodiversity?",
            "What is sustainable development?",
            "Explain the carbon cycle",
            "What are renewable energy sources?",
            "How can we reduce plastic pollution?"
        ],
        inputs=msg,
        label="Example Questions"
    )

    # Event handlers
    def respond(message, chat_history, api_key):
        return chat_with_bot(message, chat_history, api_key)

    msg.submit(respond, [msg, chatbot, api_key_input], [chatbot]).then(
        lambda: "", None, msg
    )
    send_btn.click(respond, [msg, chatbot, api_key_input], [chatbot]).then(
        lambda: "", None, msg
    )
    clear.click(lambda: [], None, chatbot)

# Launch the interface
if __name__ == "__main__":
    demo.launch(debug=True)

/tmp/ipython-input-2367013747.py:125: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2367013747.py:159: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2367013747.py:159: DeprecationWarning: The 'show_copy_button' parameter will be removed in Gradio 6.0. You will need to use 'buttons=["copy"]' instead.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2367013747.py:159: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c2584c3927d374d8e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
